In [1]:
!pip install ctgan


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.4 MB/s eta 0:00:0000:01


In [4]:
import pandas as pd
df = pd.read_csv('/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv')

In [5]:
df.head(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [7]:
print(df.dtypes)


step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object


In [12]:
import pandas as pd
from ctgan import CTGAN
from sklearn.preprocessing import LabelEncoder

# Load your data
df = pd.read_csv('/kaggle/input/paysim1/PS_20174392719_1491204439457_log.csv')
df = df.iloc[:5000, :]

# Drop the 'nameOrig' column if it should not be included
df = df.drop(columns=['nameOrig'], errors='ignore')  # Use errors='ignore' to avoid error if column doesn't exist
df = df.drop(columns=['nameDest'], errors='ignore')

# Check and convert data types if necessary
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype(str)  # Convert object columns to string

# Identify categorical columns
categorical_columns = ['type']  # Add other categorical columns if necessary

# Initialize a dictionary to store label encoders for each categorical column
label_encoders = {}

# Replace each unique categorical value with a number (label encoding)
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save the encoder to revert later

# Identify numerical columns
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Initialize CTGAN model
ctgan = CTGAN()

# Fit the model on the label-encoded data
ctgan.fit(df, epochs=300)

# Generate synthetic data
synthetic_data = ctgan.sample(1000)

# Revert the label encoding back to the original categorical text
for col in categorical_columns:
    le = label_encoders[col]
    synthetic_data[col] = le.inverse_transform(synthetic_data[col].astype(int))

# Save the generated data to an Excel file
synthetic_data.to_excel('generated_data_with_gan.xlsx', index=False)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            5000 non-null   int64  
 1   type            5000 non-null   int64  
 2   amount          5000 non-null   float64
 3   oldbalanceOrg   5000 non-null   float64
 4   newbalanceOrig  5000 non-null   float64
 5   oldbalanceDest  5000 non-null   float64
 6   newbalanceDest  5000 non-null   float64
 7   isFraud         5000 non-null   int64  
 8   isFlaggedFraud  5000 non-null   int64  
dtypes: float64(5), int64(4)
memory usage: 351.7 KB


In [14]:
df=pd.read_excel("/kaggle/working/generated_data_with_gan.xlsx")

In [15]:
df.head(20)

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,3,PAYMENT,10141.586672,9.730824e+04,2.824593e+04,5.091929e+03,1.745921e+04,0,0
1,1,PAYMENT,9520.971732,4.523153e+04,1.152787e+04,-1.426548e+04,3.059394e+02,0,0
2,1,CASH_IN,392383.326471,1.178937e+04,7.040773e+03,4.367074e+05,4.288561e+04,0,0
3,2,CASH_OUT,31105.655418,7.818787e+06,8.818796e+06,1.380479e+06,5.336523e+04,0,0
4,4,CASH_OUT,87250.423974,-3.733140e+04,-2.100470e+04,3.342905e+04,1.302768e+06,0,0
5,3,TRANSFER,3810.840617,-1.077427e+04,-1.790905e+04,3.172935e+05,3.434464e+04,0,0
6,2,PAYMENT,9839.176309,3.989938e+04,5.562584e+04,-5.145936e+04,-4.680304e+04,0,0
7,4,PAYMENT,-1431.816451,5.031102e+04,6.299940e+05,-4.723298e+04,-3.588769e+04,0,0
8,4,DEBIT,7326.050313,1.210698e+04,7.915887e+03,2.813590e+04,-1.398552e+03,0,0
9,4,PAYMENT,1737.354434,-1.778027e+04,1.866354e+04,1.652823e+04,1.111516e+04,0,0
